---

_You are currently looking at **version 1.0** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-machine-learning/resources/bANLa) course resource._

---

## Assignment 4 - Understanding and Predicting Property Maintenance Fines

This assignment is based on a data challenge from the Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)). 

The Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)) and the Michigan Student Symposium for Interdisciplinary Statistical Sciences ([MSSISS](https://sites.lsa.umich.edu/mssiss/)) have partnered with the City of Detroit to help solve one of the most pressing problems facing Detroit - blight. [Blight violations](http://www.detroitmi.gov/How-Do-I/Report/Blight-Complaint-FAQs) are issued by the city to individuals who allow their properties to remain in a deteriorated condition. Every year, the city of Detroit issues millions of dollars in fines to residents and every year, many of these fines remain unpaid. Enforcing unpaid blight fines is a costly and tedious process, so the city wants to know: how can we increase blight ticket compliance?

The first step in answering this question is understanding when and why a resident might fail to comply with a blight ticket. This is where predictive modeling comes in. For this assignment, your task is to predict whether a given blight ticket will be paid on time.

All data for this assignment has been provided to us through the [Detroit Open Data Portal](https://data.detroitmi.gov/). **Only the data already included in your Coursera directory can be used for training the model for this assignment.** Nonetheless, we encourage you to look into data from other Detroit datasets to help inform feature creation and model selection. We recommend taking a look at the following related datasets:

* [Building Permits](https://data.detroitmi.gov/Property-Parcels/Building-Permits/xw2a-a7tf)
* [Trades Permits](https://data.detroitmi.gov/Property-Parcels/Trades-Permits/635b-dsgv)
* [Improve Detroit: Submitted Issues](https://data.detroitmi.gov/Government/Improve-Detroit-Submitted-Issues/fwz3-w3yn)
* [DPD: Citizen Complaints](https://data.detroitmi.gov/Public-Safety/DPD-Citizen-Complaints-2016/kahe-efs3)
* [Parcel Map](https://data.detroitmi.gov/Property-Parcels/Parcel-Map/fxkw-udwf)

___

We provide you with two data files for use in training and validating your models: train.csv and test.csv. Each row in these two files corresponds to a single blight ticket, and includes information about when, why, and to whom each ticket was issued. The target variable is compliance, which is True if the ticket was paid early, on time, or within one month of the hearing data, False if the ticket was paid after the hearing date or not at all, and Null if the violator was found not responsible. Compliance, as well as a handful of other variables that will not be available at test-time, are only included in train.csv.

Note: All tickets where the violators were found not responsible are not considered during evaluation. They are included in the training set as an additional source of data for visualization, and to enable unsupervised and semi-supervised approaches. However, they are not included in the test set.

<br>

**File descriptions** (Use only this data for training your model!)

    train.csv - the training set (all tickets issued 2004-2011)
    test.csv - the test set (all tickets issued 2012-2016)
    addresses.csv & latlons.csv - mapping from ticket id to addresses, and from addresses to lat/lon coordinates. 
     Note: misspelled addresses may be incorrectly geolocated.

<br>

**Data fields**

train.csv & test.csv

    ticket_id - unique identifier for tickets
    agency_name - Agency that issued the ticket
    inspector_name - Name of inspector that issued the ticket
    violator_name - Name of the person/organization that the ticket was issued to
    violation_street_number, violation_street_name, violation_zip_code - Address where the violation occurred
    mailing_address_str_number, mailing_address_str_name, city, state, zip_code, non_us_str_code, country - Mailing address of the violator
    ticket_issued_date - Date and time the ticket was issued
    hearing_date - Date and time the violator's hearing was scheduled
    violation_code, violation_description - Type of violation
    disposition - Judgment and judgement type
    fine_amount - Violation fine amount, excluding fees
    admin_fee - $20 fee assigned to responsible judgments
state_fee - $10 fee assigned to responsible judgments
    late_fee - 10% fee assigned to responsible judgments
    discount_amount - discount applied, if any
    clean_up_cost - DPW clean-up or graffiti removal cost
    judgment_amount - Sum of all fines and fees
    grafitti_status - Flag for graffiti violations
    
train.csv only

    payment_amount - Amount paid, if any
    payment_date - Date payment was made, if it was received
    payment_status - Current payment status as of Feb 1 2017
    balance_due - Fines and fees still owed
    collection_status - Flag for payments in collections
    compliance [target variable for prediction] 
     Null = Not responsible
     0 = Responsible, non-compliant
     1 = Responsible, compliant
    compliance_detail - More information on why each ticket was marked compliant or non-compliant


___

## Evaluation

Your predictions will be given as the probability that the corresponding blight ticket will be paid on time.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC). 

Your grade will be based on the AUC score computed for your classifier. A model which with an AUROC of 0.7 passes this assignment, over 0.75 will recieve full points.
___

For this assignment, create a function that trains a model to predict blight ticket compliance in Detroit using `train.csv`. Using this model, return a series of length 61001 with the data being the probability that each corresponding ticket from `test.csv` will be paid, and the index being the ticket_id.

Example:

    ticket_id
       284932    0.531842
       285362    0.401958
       285361    0.105928
       285338    0.018572
                 ...
       376499    0.208567
       376500    0.818759
       369851    0.018528
       Name: compliance, dtype: float32

In [7]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import GradientBoostingClassifier

#from sklearn.linear_model import LogisticRegression
#from sklearn.svm import SVC

def blight_model():
    
    # Your code here
    # load data 
    train_df = pd.read_csv('train.csv', encoding = 'ISO-8859-1')
    test_df = pd.read_csv('test.csv', encoding = 'ISO-8859-1')
    addresses_df =  pd.read_csv('addresses.csv')
    latlons_df = pd.read_csv('latlons.csv')
    
    # start prepping data 
    #print(train_df['compliance'])
    train_df = train_df[np.isfinite(train_df['compliance'])] # drop 'not responsible' records
    #print(train_df['compliance'])
    
    # join addresses & latlons
    #print(addresses_df)
    #print(latlons_df)
    addresses_df = addresses_df.join(latlons_df.set_index('address'), on='address', how='left')
    #print(addresses_df)
    train_df = train_df.join(addresses_df.set_index('ticket_id'), on = 'ticket_id', how='left')
    test_df = test_df.join(addresses_df.set_index('ticket_id'), on = 'ticket_id', how='left')
    #print(train_df.shape)
    #print(test_df.shape)
    #print(train_df.state)
    # get rid of training records with invalid lat/lon - note - can't do this as this removes some values in test set
    #train_df = train_df[np.isfinite(train_df['lon'])]
    #print(train_df.shape)
    #test_df = test_df[np.isfinite(test_df['lon'])]
    #print(test_df.agency_name)
    #print(test_df.lat.shape)
    #print(train_df.columns)
    #print(test_df.columns)
    
    # clean up invalid lat / lon
    train_df['lat'].fillna(method='pad', inplace=True)
    train_df['lon'].fillna(method='pad', inplace=True)
    train_df['state'].fillna(method='pad', inplace=True)
    test_df['lat'].fillna(method='pad', inplace=True)
    test_df['lon'].fillna(method='pad', inplace=True)
    test_df['state'].fillna(method='pad', inplace=True)
    
    # set dummys for state / agency / disposition
    train_df = pd.get_dummies(train_df, columns=['state', 'agency_name', 'disposition'])
    test_df = pd.get_dummies(test_df, columns=['state', 'agency_name', 'disposition'])
    
    #print(test_df.columns)
    
    # set index for dataframes to ticketID
    train_df = train_df.set_index('ticket_id')
    test_df = test_df.set_index('ticket_id')
    
    # remove undesirable columns from train and set up training set 
    y_train = train_df['compliance']
    train_df = train_df.drop(['payment_status', 'compliance_detail', 'payment_amount', 'balance_due', 'payment_date', 'collection_status', 'compliance'], axis=1)
    
    # remove columns from both datasets 
    dropped_columns = ['country', 'inspector_name', 'violation_code', 'mailing_address_str_name', 'hearing_date', 'mailing_address_str_number', 'violation_description', 'city', 'non_us_str_code', 'violation_street_number', 'grafitti_status', 'ticket_issued_date', 'zip_code', 'violation_zip_code', 'violation_street_name', 'violator_name', 'address']
    train_df = train_df.drop(dropped_columns, axis=1)
    test_df = test_df.drop(dropped_columns, axis=1)
    #print(addresses_df.columns) # Index(['ticket_id', 'address'], dtype='object')
    #print(latlons_df.columns) # Index(['address', 'lat', 'lon'], dtype='object')
    #print(addresses_df.shape)
    #print(latlons_df.shape)
    # show differences in columns
    #print(set(test_df.columns) - set(train_df.columns)) # {'payment_amount', 'payment_status', 'compliance', 'balance_due', 'collection_status', 'compliance_detail', 'payment_date'}
    #print(type(compliance_series))
    #print(train_df.fine_amount)
    
    # remove columns in train_df not in test_df
    #print(list(set(train_df.columns) - set(test_df.columns)))
    train_df = train_df.drop(list(set(train_df.columns) - set(test_df.columns)), axis=1)
    test_df = test_df.drop(list(set(test_df.columns) - set(train_df.columns)), axis=1)
    #print(train_df.shape)
    #print(test_df.shape)
    #print(train_df)
    
    # try logistic w/o minmax
    #print(test_df)
    # scale the data and set up test data 
    '''
    scaler = MinMaxScaler()
    X_train = scaler.fit_transform(train_df)
    X_test = scaler.transform(test_df)
    '''
    X_train = train_df
    X_test = test_df
    #print(X_train.columns)
    #print(X_test[0])
    # create classifier 
    clf = GradientBoostingClassifier().fit(X_train, y_train)
    test_proba = clf.predict_proba(X_test)[:,1]
    #print(test_proba)
    test_df['compliance'] = test_proba
    #print(test_df['compliance'].shape)
    #clf = LogisticRegression(C=100).fit(X_train, y_train)
    #clf = SVC(kernel = 'rbf', gamma=1.0, random_state = 0).fit(X_train, y_train)
    #print(y_train)
    
    return test_df['compliance']# Your answer here

#blight_model()

/opt/conda/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2827: DtypeWarning: Columns (11,12,31) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


ticket_id
284932    0.045611
285362    0.014887
285361    0.049914
285338    0.069187
285346    0.072943
285345    0.069187
285347    0.072521
285342    0.900437
285530    0.016843
284989    0.032784
285344    0.070657
285343    0.022333
285340    0.022954
285341    0.072521
285349    0.072943
285348    0.069187
284991    0.032784
285532    0.024136
285406    0.018081
285001    0.034496
285006    0.027784
285405    0.019158
285337    0.016349
285496    0.053345
285497    0.048510
285378    0.019179
285589    0.025474
285585    0.044757
285501    0.053662
285581    0.014904
            ...   
376367    0.018258
376366    0.029214
376362    0.162737
376363    0.172707
376365    0.018258
376364    0.029214
376228    0.043055
376265    0.027264
376286    0.243807
376320    0.030180
376314    0.026535
376327    0.345222
376385    0.272408
376435    0.894638
376370    0.931224
376434    0.061545
376459    0.059342
376478    0.014679
376473    0.029327
376484    0.026453
376482    0.025185
37